# Image viewer

This notebook is for inspecting timelapse microscopy data, with associated sinhgle-cell labels and tracks, showing the infection of human macrophages with Mycobacterium Tuberculosis (Mtb), acquired on an Opera Phenix confocal microscope. 

In [1]:
import napari
from macrohet import dataio, tile
import btrack
import os

### Load experiment of choice

The Opera Phenix is a high-throughput confocal microscope that acquires very large 5-dimensional (TCZXY) images over several fields of view in any one experiment. Therefore, a lazy-loading approach is chosen to mosaic, view and annotate these images. This approach depends upon Dask and DaskFusion. The first step is to load the main metadata file (typically called `Index.idx.xml` and located in the main `Images` directory) that contains the image filenames and associated TCXZY information used to organise the images.

In [6]:
base_dir = '/mnt/DATA/sandbox/pierre_live_cell_data/outputs/Replication_IPSDM_GFP/'
# base_dir = '/Volumes/lab-gutierrezm/home/users/dayn/macrohet/'
metadata_fn = os.path.join(base_dir, 'macrohet_images/Index.idx.xml')
metadata = dataio.read_harmony_metadata(metadata_fn)  
metadata

Reading metadata XML file...
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3397, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_1359450/3511379798.py", line 4, in <cell line: 4>
    metadata = dataio.read_harmony_metadata(metadata_fn)
  File "/home/dayn/analysis/macrohet/macrohet/dataio.py", line 33, in read_harmony_metadata
FileNotFoundError: [Errno 2] No such file or directory: '/mnt/DATA/sandbox/pierre_live_cell_data/outputs/Replication_IPSDM_GFP/macrohet_images/Index.idx.xml'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 1992, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/IPython/core/ultratb.py", line 1118, in structured

### View assay layout and mask information (optional)

The Opera Phenix acquires many time lapse series from a range of positions. The first step is to inspect the image metadata, presented in the form of an `Assaylayout/experiment_ID.xml` file, to show which positions correspond to which experimental assays.

In [ ]:
metadata_path = os.path.join(base_dir, 'macrohet_images/Assaylayout/20210602_Live_cell_IPSDMGFP_ATB.xml')
assay_layout = dataio.read_harmony_metadata(metadata_path, assay_layout=True,)# mask_exist=True,  image_dir = image_dir, image_metadata = metadata)
assay_layout

### Define row and column of choice

In [ ]:
row = 3
column = 5

### Now to lazily mosaic the images using Dask prior to viewing them.

1x (75,2,3) [TCZ] image stack takes approximately 1 minute to stitch together, so only load the one field of view I want.

In [37]:
image_dir = os.path.join(base_dir, 'macrohet_images/Images_8bit')
images = tile.compile_mosaic(image_dir, 
                             metadata, 
                             row, column, 
                             set_plane='sum_proj',
#                              set_channel=1,
                             set_time = 66,
#                             input_transforms = [input_transforms]
                            )#.compute().compute()

In [38]:
%%time
images = images.compute().compute()

CPU times: user 1min 34s, sys: 9.86 s, total: 1min 44s
Wall time: 38 s


In [14]:
%%time
images = images.compute().compute()

CPU times: user 1min 59s, sys: 1min 36s, total: 3min 36s
Wall time: 57.3 s


# Load labels

In [21]:
labels_fn = os.path.join(base_dir, f"labels/full_localisation/{row,column}_objs_masks_tracks.h5") 
with btrack.io.HDF5FileHandler(labels_fn, 'r') as hdf:
    tracks = hdf.tracks
    segmentation = hdf.segmentation
    objects = hdf.objects
napari_tracks, properties, graph = btrack.utils.tracks_to_napari(tracks, ndim=2)

[INFO][2023/04/17 01:28:50 PM] Opening HDF file: /Volumes/lab-gutierrezm/home/users/dayn/macrohet/labels/full_localisation/(3, 5)_objs_masks_tracks.h5...
[INFO][2023/04/17 01:28:50 PM] Loading tracks/obj_type_1
[INFO][2023/04/17 01:28:50 PM] Loading LBEP/obj_type_1
[INFO][2023/04/17 01:28:50 PM] Loading objects/obj_type_1 (31313, 5) (31313 filtered: None)
[INFO][2023/04/17 01:29:20 PM] Loading segmentation (75, 6048, 6048)
[INFO][2023/04/17 01:29:20 PM] Loading objects/obj_type_1 (31313, 5) (31313 filtered: None)
[INFO][2023/04/17 01:29:20 PM] Closing HDF file: /Volumes/lab-gutierrezm/home/users/dayn/macrohet/labels/full_localisation/(3, 5)_objs_masks_tracks.h5


### And load downscale (ds) version

In [23]:
ds_labels_fn = os.path.join(base_dir, f"labels/full_localisation/downscaled/{row,column}_downsc_objs_masks_tracks.h5") 
with btrack.io.HDF5FileHandler(labels_fn, 'r') as hdf:
    ds_tracks = hdf.tracks
    ds_segmentation = hdf.segmentation
    ds_objects = hdf.objects
ds_napari_tracks, properties, graph = btrack.utils.tracks_to_napari(tracks, ndim=2)

[INFO][2023/04/17 01:30:21 PM] Opening HDF file: /Volumes/lab-gutierrezm/home/users/dayn/macrohet/labels/full_localisation/(3, 5)_objs_masks_tracks.h5...
[INFO][2023/04/17 01:30:21 PM] Loading tracks/obj_type_1
[INFO][2023/04/17 01:30:21 PM] Loading LBEP/obj_type_1
[INFO][2023/04/17 01:30:22 PM] Loading objects/obj_type_1 (31313, 5) (31313 filtered: None)
[INFO][2023/04/17 01:30:47 PM] Loading segmentation (75, 6048, 6048)
[INFO][2023/04/17 01:30:47 PM] Loading objects/obj_type_1 (31313, 5) (31313 filtered: None)
[INFO][2023/04/17 01:30:47 PM] Closing HDF file: /Volumes/lab-gutierrezm/home/users/dayn/macrohet/labels/full_localisation/(3, 5)_objs_masks_tracks.h5


# Launch napari image viewer

In [34]:
viewer = napari.view_image(images, rgb=True)


NameError: name 'viewer' is not defined

In [35]:
viewer = napari.view_image(images, 
                           channel_axis=1,
                           name=["macrophage", "mtb"],
                           colormap=["green",  "magenta"],
                           contrast_limits=[[100, 6000], [100, 2000]]
                           )
viewer.add_labels(segmentation, 
                  #scale=(10, 1, 1,), 
#                   color={1:'yellow'}
                 )
viewer.add_tracks(napari_tracks,
#                     properties=properties, 
#                     graph=graph, 
#                     name="Tracks [5,4]", 
                    blending="translucent",
                    visible=True,
#                     scale = (100,1,1)
                )
viewer.add_labels(ds_segmentation, 
                  #scale=(10, 1, 1,), 
#                   color={1:'yellow'}
                 )
viewer.add_tracks(ds_napari_tracks,
#                     properties=properties, 
#                     graph=graph, 
#                     name="Tracks [5,4]", 
                    blending="translucent",
                    visible=True,
#                     scale = (100,1,1)
                )

ValueError: A distribution name is required.

# Fixing duplicate localisation bug

When tracking on upscaled images, there appears to be multiple localisations for each cell.

In [35]:
objects[0].t

0

In [66]:
objects[0]

{'ID': 0, 'x': 361.4163818359375, 'y': 82.76036834716797, 'z': 0.0, 't': 0, 'dummy': False, 'states': 7, 'label': 5, 'area': 21045.0, 'major_axis_length': 214.41383, 'mean_intensity-0': 1194.5138, 'mean_intensity-1': 344.16592, 'minor_axis_length': 129.83961, 'orientation': 0.75574046}

In [36]:
# isolate objects in first frame
objs_t0 = [o for o in objects if o.t == 0]

In [54]:
# isolate objects in second frame
objs_t1 = [o for o in objects if o.t == 1]

In [55]:
len(objs_t0), len(objs_t1)

(482, 491)

In [40]:
import matplotlib.pyplot as plt
import seaborn as sns

In [43]:
x_coords = [o.x for o in objs_t0]
y_coords = [o.y for o in objs_t0]
sns.scatterplot(x = x_coords, y = y_coords)

<AxesSubplot:>

In [56]:
x_coords = [o.x for o in objs_t1]
y_coords = [o.y for o in objs_t1]
sns.scatterplot(x = x_coords, y = y_coords)

<AxesSubplot:>

In [57]:
xy_coords = [[y, x] for x, y in zip(x_coords, y_coords)]

In [58]:
viewer.add_points(xy_coords, ndim=2)

<Points layer 'Points [1]' at 0x7fab1e4085e0>

happens at the second frame (t =1), does the tracker think the cells are dividing?

In [63]:
track_352 = [track for track in tracks if track.ID == 352][0]

In [64]:
track_352

OrderedDict([('ID', 352), ('t', [0, 0, 0, 3, 0, 5, 0, 7, 0, 0, 10, 0, 0, 13, 0, 0]), ('x', [1246.5439453125, 545.9777221679688, 257.1653137207031, 1829.9788818359375, 5833.12451171875, 1793.1766357421875, 5636.16259765625, 1817.8084716796875, 933.7398071289062, 5130.16162109375, 1772.145751953125, 3305.595458984375, 5367.18896484375, 1792.47900390625, 4483.9384765625, 3577.86669921875]), ('y', [2488.89501953125, 2432.095947265625, 2485.5966796875, 526.9984130859375, 2569.903076171875, 555.8139038085938, 2573.446044921875, 541.0459594726562, 2609.950927734375, 2765.235107421875, 528.5228271484375, 2601.489990234375, 2589.444580078125, 535.2486572265625, 2591.010009765625, 2660.624267578125]), ('z', [0.0, 0.0, 0.0, -0.9898869395256042, 0.0, 9.203656196594238, 0.0, 1.416200876235962, 0.0, 0.0, -0.5789438486099243, 0.0, 0.0, 1.7207143306732178, 0.0, 0.0]), ('parent', 352), ('root', 352), ('state', [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]), ('generation', 0), ('dummy', [False, False, False, True, False, True, False, True, False, False, True, False, False, True, False, False]), ('mean_intensity-1', array([375.27191162, 371.12649536, 381.73098755,          nan,
       527.75140381,          nan, 378.95132446,          nan,
       374.93865967, 430.74038696,          nan, 420.56481934,
       371.16625977,          nan, 380.08053589, 373.87451172])), ('minor_axis_length', array([205.80731201, 105.05467987, 168.41955566,          nan,
       166.89424133,          nan, 228.62567139,          nan,
       153.93232727, 237.12020874,          nan, 151.34983826,
       138.97561646,          nan, 148.19760132, 205.91195679])), ('orientation', array([ 1.34212172,  1.04459202, -1.25169444,         nan,  0.35254607,
               nan,  0.9975701 ,         nan, -0.30199218, -0.56135589,
               nan,  0.28361169,  0.92129934,         nan, -1.38651276,
        0.33074108])), ('area', array([82943., 12403., 40935.,    nan, 33226.,    nan, 65616.,    nan,
       37422., 94283.,    nan, 24503., 24993.,    nan, 63285., 46406.])), ('major_axis_length', array([530.6784668 , 151.08787537, 343.18338013,          nan,
       267.50311279,          nan, 410.57269287,          nan,
       323.98602295, 549.95996094,          nan, 207.88874817,
       231.50190735,          nan, 559.74267578, 299.54721069])), ('mean_intensity-0', array([1331.94360352, 3039.46801758, 2464.49560547,           nan,
       1702.59118652,           nan, 2152.76171875,           nan,
       2121.22558594, 2459.67089844,           nan, 2424.61108398,
       1312.19689941,           nan, 1673.03894043, 2105.97827148]))])

In [61]:
track_463 = [track for track in tracks if track.ID == 463][0]

In [62]:
track_463

OrderedDict([('ID', 463), ('t', [1, 1, 2, 1, 4, 1, 6, 7, 8, 9, 10, 1, 1, 13, 1, 15, 1, 1]), ('x', [937.5814208984375, 247.6570281982422, 247.6570281982422, 1981.85498046875, 376.3756408691406, 5182.4599609375, 425.9117431640625, 468.83990478515625, 511.76806640625, 554.6962280273438, 597.6243896484375, 5462.75341796875, 4377.29052734375, 561.3092041015625, 3385.734375, 528.7427368164062, 3933.203125, 4980.35791015625]), ('y', [5157.68896484375, 2289.09814453125, 2289.09814453125, 5271.1650390625, 2251.976806640625, 5276.767578125, 2227.08203125, 2209.04736328125, 2191.0126953125, 2172.97802734375, 2154.943359375, 5198.47314453125, 5240.32421875, 2178.76611328125, 5238.8017578125, 2160.18701171875, 5326.43359375, 5389.53173828125]), ('z', [0.0, 0.0, 0.0, 0.0, 0.045263469219207764, 0.0, -1.3256895542144775, -2.651379108428955, -3.9770684242248535, -5.30275821685791, -6.628447532653809, 0.0, 0.0, -1.3980931043624878, 0.0, -5.49375581741333, 0.0, 0.0]), ('parent', 463), ('root', 463), ('state', [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]), ('generation', 0), ('dummy', [False, True, True, False, True, False, True, True, True, True, True, False, False, True, False, True, False, False]), ('mean_intensity-1', array([393.75064087,          nan,          nan, 354.84616089,
                nan, 378.23861694,          nan,          nan,
                nan,          nan,          nan, 426.65826416,
       383.91149902,          nan, 370.95663452,          nan,
       372.15176392, 426.39138794])), ('minor_axis_length', array([173.58474731,          nan,          nan, 266.72103882,
                nan, 218.0191803 ,          nan,          nan,
                nan,          nan,          nan, 228.15093994,
       210.03012085,          nan, 211.1125946 ,          nan,
       370.98345947, 235.39424133])), ('orientation', array([ 1.04202902,         nan,         nan,  0.16061994,         nan,
        0.02034526,         nan,         nan,         nan,         nan,
               nan, -1.03508258,  1.01339304,         nan,  1.20384955,
               nan, -1.23565578,  0.76379269])), ('area', array([ 48065.,     nan,     nan,  76747.,     nan,  68762.,     nan,
           nan,     nan,     nan,     nan,  48909.,  70575.,     nan,
        56456.,     nan, 130842., 117937.])), ('major_axis_length', array([366.2220459 ,          nan,          nan, 381.24386597,
                nan, 433.84262085,          nan,          nan,
                nan,          nan,          nan, 276.78848267,
       438.91143799,          nan, 395.62295532,          nan,
       478.30978394, 667.7713623 ])), ('mean_intensity-0', array([1671.66247559,           nan,           nan, 2246.65380859,
                 nan, 2224.53344727,           nan,           nan,
                 nan,           nan,           nan, 1927.07592773,
       5058.89111328,           nan, 1968.14624023,           nan,
       3394.10302734, 2807.16772461]))])

In [85]:
def track_and_show(objects, masks, config_fn, search_radius = 20, method = ["MOTION", "VISUAL"]):
    
    # initialise a tracker session using a context manager
    with btrack.BayesianTracker() as tracker:
        # configure the tracker using a config file
        tracker.configure(config_fn)
        # set max search radius
        tracker.max_search_radius = search_radius
        # define tracking method
        tracker.tracking_updates = ["MOTION", "VISUAL"]
        # redefine features so that both channels are included in track measurements
        tracker.features = list(objects[0].properties.keys())
        # append the objects to be tracked
        tracker.append(objects)
        # set the tracking volume
        tracker.volume=((0, 6048), (0, 6048))
        # track them (in interactive mode)
        tracker.track(step_size=25)
        # generate hypotheses and run the global optimizer
        tracker.optimize()
        # store the tracks
        tracks = tracker.tracks
    
    napari_tracks, properties, graph = btrack.utils.tracks_to_napari(tracks, ndim=2)
    
    viewer.add_tracks(napari_tracks, 
                      blending="translucent",
                      visible=True,)

In [86]:
track_and_show(objects, 
               segmentation, 
               '/home/dayn/analysis/btrack/models/cell_config.json', 
               5, 
               ["MOTION",]# "VISUAL"]
              )

OSError: Cannot load shared library /home/dayn/analysis/BayesianTracker/btrack/libs/libtracker.so